## Code generation

In [1]:
# import sys
# !{sys.executable} -m pip install --upgrade pip
# !{sys.executable} -m pip install sympy
# !{sys.executable} -m pip install scipy
# !{sys.executable} -m pip install matplotlib

In [2]:
# Remove matplotlib deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
# initialize sympy printing (for latex output)
from sympy import init_printing, Symbol
init_printing()

# import functions and classes for compartment models
import sys
sys.path.insert(0, "./compartor") #use local copy
from compartor import *

Here, we illustrate how to export the moment equations in LaTeX format or generate code for simulations.

We consider again the exemplary model of the paper and derive moments equations with the automated function.

In [4]:
x = Content('x')
y = Content('y')
z = Content('z')

from sympy import symbols, log, exp, sin, cos, tan
kM,h = symbols('kM h') # The Michaelis-Menten constant and hill exponent

# define intake distribution
pi_I = OutcomeDistribution.Poisson(Symbol("\pi_{Poiss}(y; \lambda)"), y[0], Symbol("\lambda"))
# define fragmentation distribution
pi_F = OutcomeDistribution.Uniform(Symbol("\pi_F(y|x)"), y[0], 0, x[0])

Birth         = TransitionClass( [x]       -to> [x+1],       'k_b',                  name='b')

Death         = TransitionClass( [x]       -to> [x-1],       'k_d', g=x[0],    name='d')
# Death         = TransitionClass( [x]       -to> [x-2],       'k_d', g=x[0]*(x[0]-1)/2,    name='d')
# Death         = TransitionClass( [x]       -to> [x-3],       'k_d', g=x[0]*(x[0]-1)*(x[0]-2)/6,    name='d')
# Death         = TransitionClass( [x]       -to> [x-1],       'k_d', g=x[0]/(x[0]+kM),    name='d')

Intake        = TransitionClass( {}        -to> [y],         'k_I',         pi=pi_I, name='I')
Exit          = TransitionClass( [x]       -to> {},          'k_E',                  name='E')

Coagulation   = TransitionClass( [x] + [y] -to> [x+y],       'k_C',                  name='C')
# Coagulation   = TransitionClass( [x] + [y] -to> [x+y],       'k_C', g=x[0]+y[0],     name='C')
# Coagulation   = TransitionClass( [x] + [y] -to> [x+y],       'k_C', g=(x[0]+y[0])**h/( (x[0]+y[0])**h + kM**h),     name='C')
# Coagulation   = TransitionClass( [x] + [y] -to> [x+y],       'k_C', g=x[0]**2+y[0]**2,     name='C')
# Coagulation   = TransitionClass( [x] + [y] -to> [x+y],       'k_C', g=(x[0]+y[0])**2,     name='C')
# Coagulation   = TransitionClass( [x] + [y] -to> [x+y],       'k_C', g=log(1+x[0]+y[0]),     name='C')
# Coagulation   = TransitionClass( [x] + [y] -to> [x+y],       'k_C', g=1+sin(x[0]+y[0]),     name='C')

# Fragmentation = TransitionClass( [x]       -to> [y] + [x-y], 'k_F', g=x[0], pi=pi_F, name='F')
# Fragmentation = TransitionClass( [x]       -to> [y] + [x-y], 'k_F', g=x[0]/(x[0]+kM), pi=pi_F, name='F')
Fragmentation = TransitionClass( [x]       -to> [y] + [x-y], 'k_F', g=x[0]*(x[0]-1)/2, pi=pi_F, name='F')

transitions = [Intake, Exit, Coagulation, Fragmentation, Birth, Death]
display_transition_classes(transitions)

                     EmptySet ---> [y],  h_I = \pi_{Poiss}(y; \lambda)*k_I                      
                               [x] ---> EmptySet,  h_E = k_E*n(x)                               
[x] + [y] ---> [x + y],  h_C = k_C*(n(y) - KroneckerDelta(x, y))*n(x)/(KroneckerDelta(x, y) + 1)
              [x] ---> [x - y] + [y],  h_F = \pi_F(y|x)*k_F*(x[0] - 1)*n(x)*x[0]/2              
                              [x] ---> [(1,) + x],  h_b = k_b*n(x)                              
                           [x] ---> [(-1,) + x],  h_d = k_d*n(x)*x[0]                           

In [5]:
moments = [
    Moment(0),
    Moment(0)**2,
    Moment(1),
    Moment(1)**2
]
display(moments)
equations = automated_moment_equations(1, transitions, moments, clna=True)
# equations = automated_moment_equations(1, transitions, moments, clna=False)
display_moment_equations(equations)

⎡                 2                      2   ⎤
⎣Moment(0), Moment (0), Moment(1), Moment (1)⎦

Computed moment equations for desired moments $\left< N\right> $, $\left< N^{2}\right> $, $\left< M^{1}\right> $, and $\left< {\left(M^{1}\right)}^{\mathtt{\text{2}}}\right> $.

Equations were iteratively added for $\left< N M^{1}\right> $.

                                      2                                                                                            2                 
d                      k_C⋅Expectation (N)   k_C⋅Expectation(N)                        k_F⋅Expectation(Moment(1))   k_F⋅Expectation (Moment(1))      
──(Expectation(N)) = - ─────────────────── + ────────────────── - k_E⋅Expectation(N) - ────────────────────────── + ─────────────────────────── + k_I
dt                              2                    2                                             2                      2⋅Expectation(N)           

                                                           2                                                                                                                     
d ⎛           ⎛ 2⎞⎞                  3      k_C⋅Expectation (N)                                   ⎛ 2⎞   k_C⋅Expectation(N)                  ⎛ 2⎞                                
──⎝Expectation⎝N ⎠⎠ = k_C⋅Expectation (N) + ─────────────────── - 2⋅k_C⋅Expectation(N)⋅Expectation⎝N ⎠ - ────────────────── + k_C⋅Expectation⎝N ⎠ + k_E⋅Expectation(N) - 2⋅k_E⋅Ex
dt                                                   2                                                           2                                                               
                                                                                                                                                                                 

                                                                                                             

d                                                                                                                      
──(Expectation(Moment(1))) = \lambda⋅k_I - k_E⋅Expectation(Moment(1)) + k_b⋅Expectation(N) - k_d⋅Expectation(Moment(1))
dt                                                                                                                     

                                                                                                                                                 2                               
d ⎛           ⎛      2   ⎞⎞          2                                                                             ⎛      2   ⎞   k_E⋅Expectation (Moment(1))                    
──⎝Expectation⎝Moment (1)⎠⎠ = \lambda ⋅k_I + 2⋅\lambda⋅k_I⋅Expectation(Moment(1)) + \lambda⋅k_I - 2⋅k_E⋅Expectation⎝Moment (1)⎠ + ─────────────────────────── + k_b⋅Expectation(N
dt                                                                                                                                       Expectation(N)                          

                                                                                               
                                                                                   ⎛      2   ⎞
) + 2⋅k_b⋅Expectation(N⋅Moment(1)) + k_d⋅Expectation(Moment(1)) - 2⋅k_d⋅Expectation⎝Moment (1)⎠

                                                                                         2                                                                                       
d                                                                         k_C⋅Expectation (N)⋅Expectation(Moment(1))                                                 k_C⋅Expectat
──(Expectation(N⋅Moment(1))) = \lambda⋅k_I⋅Expectation(N) + \lambda⋅k_I + ────────────────────────────────────────── - k_C⋅Expectation(N)⋅Expectation(N⋅Moment(1)) + ────────────
dt                                                                                            2                                                                                  
                                                                                                                                                                                 

                                                                                                ⎛      2   ⎞ 

The LaTeX source of the ODE system can be found and copy-pasted by modifying the Math Render option of the jupyter notebook. This is done by right-clicking on the system and choosing Math Settings > Math Render > Plain Source .

From the closed equations, we can also generate code to simulate the system. Currently, Python or Julia code can be generated.

For a direct code output, the user can rely on the functions:

In [6]:
# python_code = generate_python_code(equations, function_name="coagulation_fragmentation")
# print(python_code)
# exec(python_code) 

In [7]:
julia_code = generate_julia_code(equations, function_name="IECFBDq")
print(julia_code)

# evaluate ODEs
function IECFBDq_ODEs(dM, M, parameters, t)
  c3 = parameters[:\lambda] # \lambda
  c6 = parameters[:kC] # kC
  c0 = parameters[:kE] # kE
  c2 = parameters[:kF] # kF
  c4 = parameters[:kI] # kI
  c5 = parameters[:kb] # kb
  c1 = parameters[:kd] # kd
  # Number of Compartments (N)
  dM[1] = c4+1/2*c6*M[1]-c0*M[1]-1/2*c6*M[1]^2-1/2*c2*M[3]+1/2*c2*M[3]^2/M[1]
  # N^2
  dM[2] = c4+c6*M[1]^3+c6*M[2]+c0*M[1]+1/2*c6*M[1]^2-c2*M[5]-2*c0*M[2]+2*c4*M[1]-1/2*c6*M[1]-1/2*c2*M[3]+1/2*c2*M[3]^2/M[1]-2*c6*M[2]*M[1]-c2*M[3]^2*M[2]/M[1]^2+2*c2*M[5]*M[3]/M[1]
  # Total Mass
  dM[3] = c3*c4+c5*M[1]-c0*M[3]-c1*M[3]
  # M^2
  dM[4] = c3*c4+c4*c3^2+c5*M[1]+c1*M[3]-2*c0*M[4]-2*c1*M[4]+2*c5*M[5]+c0*M[3]^2/M[1]+2*c3*c4*M[3]
  # N*M
  dM[5] = c3*c4+c0*M[3]+c4*M[3]+c5*M[2]+1/2*c6*M[5]-c1*M[5]-2*c0*M[5]-1/2*c2*M[4]+c3*c4*M[1]+1/2*c6*M[1]^2*M[3]-c6*M[5]*M[1]+c2*M[4]*M[3]/M[1]-1/2*c2*M[3]^2*M[5]/M[1]^2
  return
end

# initialize expected moments vector
function IECFBDq_initial(n0)
  M = zeros(5)
  #